<a href="https://colab.research.google.com/github/jinnyfruit/Data_Science_Project/blob/main/PredictScore_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import random

#before predict target values through model, preprocess dataframe
def prepare_df(df_original):
  #create a dataframe without unnecessary features 
  df=df_original.drop(['Stress busters','Time utilized','Do you find yourself more connected with your family, close friends , relatives  ?'],axis=1)
  df=(df.drop('Time spent on TV',axis=1).join(df['Time spent on TV'].apply(pd.to_numeric,errors='coerce')))
  df=df.replace({'Excellent':5,'Good':4,'Average':3,'Poor':2,'Very poor':1})
  df.dropna(inplace=True)
  #seperate taret values
  #X=df.iloc[:,[2,3,6,7,8,9,10,13]]
  grid=[2,3,6,7,8,9,10,13]
 
  # sample features without duplication
  alist=[]        
  count=random.randint(1,8)  
  for i in range(count):
    a = random.randint(0,7)       
    while a in alist : 
      a = random.randint(0,7)
    alist.append(grid[a]) #store features in alist

  randX=df.iloc[:,alist]  #create X with extracted features 
  y=df['Rating of Online Class experience'].values
  return df,randX,y

#return n features according to its importance
def feature_importance(X,y,n):
  #feature importance scoring
  from sklearn.ensemble import ExtraTreesClassifier
  model=ExtraTreesClassifier()
  model.fit(X,y)
  print(model.feature_importances_[0:n])
  feat_importances=pd.Series(model.feature_importances_,index=X.columns)
  feat_importances.nlargest(n).plot(kind='barh')
  plt.show()
  return feat_importances.nlargest(n)

#predict target value using KNN and KFold
def predictScore(df,kfold_n):
  kfold=KFold()
  df,X,y=prepare_df(df)
  score=[]
  if(kfold_n<0):
    return 0
  for train, test in kfold.split(df):
    temp=[]
    X_train,X_test=X.iloc[train,:],X.iloc[test,:]
    y_train,y_test=y[train],y[test]
  
    knn=KNeighborsClassifier(n_neighbors=(kfold_n+1))
    knn.fit(X_train,y_train)
    knn.predict(X_test)
    temp.append(knn.score(X_test,y_test))
  score.append(np.max(temp))
  return list(X.columns),np.max(score)


df=pd.read_csv('/content/COVID-19 Survey Student Responses (1).csv')
for i in range(1,20):
  print(predictScore(df,kfold_n=i))


(['Age of Subject', 'Time spent on TV', 'Time spent on social media', 'Time spent on sleep', 'Time spent on fitness', 'Age of Subject', 'Time spent on sleep'], 0.3452914798206278)
(['Time spent on fitness'], 0.25112107623318386)
(['Time spent on self study', 'Time spent on TV', 'Time spent on social media', 'Time spent on TV', 'Time spent on self study'], 0.26905829596412556)
(['Number of meals per day'], 0.25112107623318386)
(['Time spent on self study', 'Time spent on TV', 'Time spent on sleep', 'Time spent on sleep'], 0.29596412556053814)
(['Time spent on self study', 'Time spent on TV', 'Time spent on social media', 'Time spent on fitness', 'Time spent on Online Class', 'Time spent on social media'], 0.31390134529147984)
(['Time spent on self study', 'Time spent on self study', 'Time spent on Online Class', 'Time spent on self study', 'Time spent on sleep', 'Time spent on self study', 'Age of Subject', 'Time spent on TV'], 0.32286995515695066)
(['Time spent on TV', 'Time spent on s